## Decorator Tutorial ##

#### - Quansight (c) 2020 ####

<h3><i> What are Decorators in Python ? and How Do They Work ?</i></h3>

<i> Before we answer how decorators work in Python, lets explore / review python functions </i>

Basic Python Function (def):

In [70]:
def func1():
    pass

<i>func1</i> is a python function which is a <b>First Class Object</b> in python

<i>What does First Class Object Mean ? Functions are "First Class Citizens" in Python, have all "pythonic" features just like classes and ints, lists, and dicts</i>

In [71]:
func1.__name__  ## it has a name attribute

'func1'

In [72]:
func1.__dict__  ## it has a dict attribute

{}

In [73]:
func1  ##  def func1 enters 'func1' name in the current module name space __main__

<function __main__.func1()>

In [74]:
set(dir(func1))  ## here are all the 'dunder' hidden attributes/methods of this funciton

{'__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__'}

<h5> We can Assign attributes to functions , something which can't be done in other languages</h5>

In [75]:
func1.value = 5

In [76]:
func1.value

5

In [77]:
func1.__dict__

{'value': 5}

In [78]:
def pub_dir(func):
    return [a for a in dir(func) if not a.startswith('__')]

In [79]:
pub_dir(func1)

['value']

<h5> Since Python functions are objects we can assign another function as an attribute </h5>

In [80]:
func1.sqval = lambda x: x*x ## lambda is another way to define a function without a name, i.e. anonymous function

In [81]:
func1.sqval

<function __main__.<lambda>(x)>

In [82]:
func1.sqval(3)

9

In [83]:
pub_dir(func1)

['sqval', 'value']

<h5>Functions passing other functions as arguments (--Basic Example, not Production Code--)</h5>

In [84]:
def upcase(func):
    func1_out = func()
    return func1_out.upper()

In [85]:
def hello():
    return "hello"

In [86]:
def howdy():
    return "howdy"

In [87]:
upcase(hello)

'HELLO'

In [88]:
upcase(howdy)

'HOWDY'

<h5>Now Let's see how this can work with decorator syntax </h5>

In [89]:
@upcase
def say_hello_again():
    return "Hello Again !"

In [90]:
say_hello_again

'HELLO AGAIN !'

<h5> A decorator is another way of calling a function in python with another function as an agrument </h5>

in other words

In [91]:
from math import sqrt
def mysqroot(x):
    return sqrt(x)
    
def mysquare(x):
    return x*x

In [92]:
(mysqroot,mysquare)

(<function __main__.mysqroot(x)>, <function __main__.mysquare(x)>)

In [93]:
"""Do NOT try this at Home ! """

def back_to_self(x):
    return mysqroot(mysquare(x))

In [94]:
back_to_self(10)

10.0

is the same as 

In [97]:
from functools import wraps
def decsqroot(f):
    @wraps(f)
    def func(*args,**kwargs):
        x = f(*args,**kwargs)
        return sqrt(x)
    return func

@decsqroot
def back_to_self2(x):
    return x*x             ### Look Mom we square the X, lot's of wastful instructions

functools.wraps preserves the name of the function being decorated

In [98]:
back_to_self2

<function __main__.back_to_self2(x)>

In [99]:
back_to_self2(4)

4.0

In [100]:
@decsqroot
def to_the_third_then_sqrt(x):
    return x**3

In [101]:
to_the_third_then_sqrt(10) == sqrt(1000)

True

In [102]:
@decsqroot
def times_any_two_then_sqrt(x,y):
    return x*y

In [103]:
times_any_two_then_sqrt(25,35) == sqrt(25*35)

True

<h5>Let's take a look what's going on under covers a bit </h5>

In [104]:
import inspect

In [105]:
comp = compile(inspect.getsource(times_any_two_then_sqrt),'?','exec')

In [106]:
comp2  = compile(inspect.getsource(to_the_third_then_sqrt),'?','exec')

In [107]:
import dis

In [108]:
dis.dis(times_any_two_then_sqrt)

  5           0 LOAD_DEREF               0 (f)
              2 LOAD_FAST                0 (args)
              4 LOAD_FAST                1 (kwargs)
              6 CALL_FUNCTION_EX         1
              8 STORE_FAST               2 (x)

  6          10 LOAD_GLOBAL              0 (sqrt)
             12 LOAD_FAST                2 (x)
             14 CALL_FUNCTION            1
             16 RETURN_VALUE


Another Way to write Decorators , use a helper module !

https://wrapt.readthedocs.io/en/latest

In [66]:
#!conda install wrapt

In [67]:
import wrapt

Universal Decorator Pattern 
https://wrapt.readthedocs.io/en/latest/decorators.html?highlight=universal#universal-decorators

In [68]:
import inspect

@wrapt.decorator
def universal(wrapped, instance, args, kwargs):
    if instance is None:
        if inspect.isclass(wrapped):
            # Decorator was applied to a class.
            return wrapped(*args, **kwargs)
        else:
            # Decorator was applied to a function or staticmethod.
            return wrapped(*args, **kwargs)
    else:
        if inspect.isclass(instance):
            # Decorator was applied to a classmethod.
            return wrapped(*args, **kwargs)
        else:
            # Decorator was applied to an instancemethod.
            return wrapped(*args, **kwargs)

decsqroot -- Version 2 Revisited, now using wrapt

In [109]:
@wrapt.decorator
def decsqroot2(f,inst,args,kwargs):
    x = f(*args,**kwargs)
    return sqrt(x)

In [110]:
@decsqroot2
def times_any_two_then_sqrt2(x,y):
    return x*y

In [111]:
from math import sqrt
times_any_two_then_sqrt2(25,35) == sqrt(25*35)

True

And of course there is a class syntax to write a decorator by overriding __call__ method, I leave it for an excercise in this short tutorial

Thank you, and hope this was helpful and useful. THE END.